# Modelo

## Constantes

In [18]:
NP = (110119 + 109164 + 111111) / 3 
A_TERR = 17.32 * NP / 3000

A_SOT = 8.66 ** 2                   # m2
Q_MAX = 8                           # mm/h
D_H_MAX = 4                         # m
D_H_MIN = 1                         # m
H_S = 3.50                          # m

C_SAT = 0.90                        # admin
C_0 = 0.60                          # admin

T_K = (1-NP/140000)                 # h


# EC 1
def f_der_v(q_ent, q_sal):
    return q_ent - q_sal

# EC 2
def f_q_ent(c, i, a_terr):
    return c * i * a_terr           # admin * m/h * m2 = m3/h

# EC 3
def f_q_sal(q_max, d_h_max, d_h_min, d_h):
    return q_max * (((d_h_max - d_h)/(d_h_max - d_h_min)) ** 0.5)

# EC 4
def f_d_h(h_s, h):
    return h_s - h

# EC 5
def f_h(v, a_sot):
    return v / a_sot

# EC 6
def f_der_c(v, v_sot, t_k, c_sat, c):
    return (v/(v_sot * t_k)) * (c_sat - c)

## Métodos

In [19]:
def euler_explicito(f, h, t, u):    # m3
    return u + h * f(t, u)          # m3 + h * m3/h

def euler_explicito_2(f, h, t, u, v):       # m3
    return u + h * f(t, u, v)               # m3 + h * m3/h

def euler_explicito_3(f, h, t, u, v, w):    # m3
    return u + h * f(t, u, v, w)            # m3 + h * m3/h

## 1A

In [20]:
intensity = 241.4 / 1000                    # m/h

DUR = 1                             # h
C = 1                               # a-dimensional
Q_SAL = 0                           # m3/h

def d_volume(t, u):
    q_ent = f_q_ent(C, intensity, A_TERR)   # m3/h
    q_sal = Q_SAL                   # m3/h
    
    return f_der_v(q_ent, q_sal)    # m3/h

time_gap = 5 / 60                          # h

def main(method, func, time_gap):
    t = 0
    u = 0
    h = time_gap

    i = 0
    print(f"t_{i} = {t}hs, u_{i} = {u}m3")

    while t < DUR:
        i += 1
        u = method(func, h, t, u)
        t += h
        print(f"t_{i} = {t}hs, u_{i} = {u}m3")

main(euler_explicito, d_volume, time_gap)

t_0 = 0hs, u_0 = 0m3
t_1 = 0.08333333333333333hs, u_1 = 12.790677526962963m3
t_2 = 0.16666666666666666hs, u_2 = 25.581355053925925m3
t_3 = 0.25hs, u_3 = 38.372032580888884m3
t_4 = 0.3333333333333333hs, u_4 = 51.16271010785185m3
t_5 = 0.41666666666666663hs, u_5 = 63.95338763481482m3
t_6 = 0.49999999999999994hs, u_6 = 76.74406516177778m3
t_7 = 0.5833333333333333hs, u_7 = 89.53474268874075m3
t_8 = 0.6666666666666666hs, u_8 = 102.32542021570372m3
t_9 = 0.75hs, u_9 = 115.11609774266668m3
t_10 = 0.8333333333333334hs, u_10 = 127.90677526962965m3
t_11 = 0.9166666666666667hs, u_11 = 140.69745279659261m3
t_12 = 1.0hs, u_12 = 153.48813032355557m3


## 1B

Discretizar las ec. 1 y ec. 6 con el método de Euler, considerando $C$ y $Q_{sal}$ variables. Correr
el modelo para todas las duraciónes / intensidades de precipitación, un lapso de tiempo
suficiente como para que el sótano se vacíe

In [25]:
storms = [       # min, mm/h
    (5, 241.4),
    (10, 190.7),
    (15, 162.6),
    (30, 119.6),
    (60, 85.0),
    (180, 41.7),
    (360, 26.4),
    (720, 16.7),
    (1440, 10.9),
    (4320, 5.2),
]

# EC 1 = volumen = u, u'=f(t, u, v)
# EC 2 = coef_inf = v, v'=g(t, u, v)

def d_volume(t, u, v, i):
    c = v
    a_terr = A_TERR

    q_max = Q_MAX
    d_h_max = D_H_MAX
    d_h_min = D_H_MIN
    
    h_s = H_S
    
    a_sot = A_SOT
    h = f_h(v, a_sot)

    d_h = f_d_h(h_s, h)

    q_ent = f_q_ent(c, i, a_terr)
    q_sal = f_q_sal(q_max, d_h_max, d_h_min, d_h)


    return f_der_v(q_ent, q_sal)

def d_infiltration(t, volume, infiltration):

    sotane_volume = H_S * A_SOT

    caracteristic_time = T_K
    saturation_infiltration = C_SAT

    return f_der_c(volume, sotane_volume, caracteristic_time, saturation_infiltration, infiltration)

def main(storm_duration, intensity, d_volume, d_infiltration, time_gap):
    time = 0
    volume = 0
    infiltration = C_0

    iteration = 0

    while time < storm_duration:
        iteration += 1
        volume, infiltration = euler_explicito_3(d_volume, time_gap, time, volume, infiltration, intensity), euler_explicito_2(d_infiltration, time_gap, time, volume, infiltration)
        time += time_gap

    intensity  = 0
    while volume > 0:
        iteration += 1
        volume, infiltration = euler_explicito_3(d_volume, time_gap, time, volume, infiltration, intensity), euler_explicito_2(d_infiltration, time_gap, time, volume, infiltration)
        time += time_gap

    print(f"v_{iteration} = {round(volume, 2)}m3")

for duration_min, intensity_mm in storms:
    duration = duration_min / 60
    intensity = intensity_mm / 1000
    time_gap = duration / 10
    main(duration, intensity, d_volume, d_infiltration, time_gap)
    print("\n")

v_1939 = -0.02m3


v_3308 = -0.03m3


v_6860 = -0.04m3




TypeError: '>' not supported between instances of 'complex' and 'int'